In [17]:
#si-exercise

import random
import numpy as np
import multiprocessing as mp
import timeit

#define constants
n_REPS = 1000
n_SAMPLES = 1000

#sample normal distributions and assign to variables
def generate_data(n_samples=n_SAMPLES):
  a = np.random.normal(15, 2, n_samples)
  x1 = np.random.normal(3, 5, n_samples)
  x2 = np.random.normal(10, 1, n_samples)
  x3 = np.random.normal(8, 8, n_samples)
  e = np.random.normal(0, 1, n_samples)
  y = a + x1 + 2 * x2 + 1/2 * x3 + e
  return y

#Serial calculation
def sCalc(n_reps=n_REPS):
  results = []
  for _ in range(n_reps):
    mean_y = generate_data().mean()
    results.append(mean_y)
  return results

#Parallel calculation
def pCalc(n_reps=n_REPS):
  results = []
  with mp.Pool(processes=2) as pool:
    for _ in range(n_reps):
      y = pool.apply_async(generate_data)
      mean_y = y.get().mean()
      results.append(mean_y)
  return results

#store sCalc results
sResults = sCalc(n_reps=n_REPS)
print(sResults)

#store pCalc results
pResults = pCalc(n_reps=n_REPS)
print(pResults)

#time sCalc
sTime = []
sTime.append(timeit.Timer('sCalc(n_reps=n_REPS)',
    'from __main__ import sCalc, generate_data, n_REPS').timeit(number=1))
print(sTime)

#time pCalc
pTime = []
pTime.append(timeit.Timer('pCalc(n_reps=n_REPS)',
    'from __main__ import pCalc, generate_data, n_REPS').timeit(number=1))
print(pTime)

[42.472923491038365, 41.94625902032077, 41.530200745705024, 41.70591112240933, 42.058332242855265, 42.038739533829215, 42.03620003675932, 41.91551823983331, 41.64161145720673, 42.09975067107872, 42.22869346382069, 42.12140269908761, 41.96176585409934, 42.16901258850101, 41.79083910182014, 41.958608389562535, 41.94614483123369, 42.03122537776338, 42.04118375604333, 41.994881933425184, 41.61884633046669, 42.130725411089365, 42.39504239865151, 42.04157343871507, 41.665668491506906, 42.156944538611214, 42.10568465809463, 42.20016720871252, 41.9041481408783, 41.77711783770658, 42.04750215444126, 42.48875934051128, 41.951974211691876, 42.184303869648694, 41.80620521643017, 42.35290820147339, 41.94733054969881, 41.76588253774747, 41.69550488830186, 42.12020212697543, 42.10471433978092, 41.731729520852056, 42.0657472955622, 41.86631352920408, 42.28368122780884, 42.29875828821573, 41.93692646193538, 42.32895970792918, 41.9776574904861, 42.08477412527065, 42.04446564513369, 42.067374476511986, 4